# Setting up the endpoint

In [4]:
%%time
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import csv_serializer

role = sagemaker.session.get_execution_role()

model_data='s3://inf1-compiled-bert-model/model.tar.gz'

pytorch_model = PyTorchModel(model_data=model_data, 
                             role=role,
                             entry_point='inference.py',
                             image='249828997522.dkr.ecr.us-east-1.amazonaws.com/bert-inf1-serving:latest',
                             framework_version='1.5.0',
                             enable_cloudwatch_metrics=True)

predictor = pytorch_model.deploy(instance_type='ml.inf1.xlarge', initial_instance_count=1)



Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Your model is not compiled. Please compile your model before using Inferentia.


-------------!CPU times: user 12.3 s, sys: 1.77 s, total: 14.1 s
Wall time: 7min 3s


In [3]:
print(f'Endpoint Name: {predictor.endpoint}')

Endpoint Name: bert-inf1-serving-2020-12-11-02-17-17-879


In [4]:
import numpy as np 
import random
import datetime
import math
import time
import boto3   
import random
from transformers import AutoTokenizer
import pickle
import os
import argparse
from concurrent import futures


def numpy_bytes_serializer(data):
    f = io.BytesIO()
    np.save(f, data)
    f.seek(0)
    return f.read()

predictor.content_type = 'application/binary'
predictor.serializer = None
predictor.deserializer = None

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [5]:
# Data
sentence1 ="The animal didn't cross the street because it was too tired."

s_nouns = ["A dude", "My mom", "The king", "Some guy", "A cat with rabies", "A sloth", "Your homie", "This cool guy my gardener met yesterday", "Superman"]
p_nouns = ["These dudes", "Both of my moms", "All the kings of the world", "Some guys", "All of a cattery's cats", "The multitude of sloths living under your bed", "Your homies", "Like, these, like, all these people", "Supermen"]
s_verbs = ["eats", "kicks", "gives", "treats", "meets with", "creates", "hacks", "configures", "spies on", "retards", "meows on", "flees from", "tries to automate", "explodes"]
p_verbs = ["eat", "kick", "give", "treat", "meet with", "create", "hack", "configure", "spy on", "retard", "meow on", "flee from", "try to automate", "explode"]
infinitives = ["to make a pie.", "for no apparent reason.", "because the sky is green.", "for a disease.", "to be able to make toast explode.", "to know more about archeology."]


sentence_random = (random.choice(s_nouns) + ' ' + random.choice(s_verbs) + ' ' + random.choice(s_nouns).lower() or random.choice(p_nouns).lower() + ' ' + random.choice(infinitives))
encoded_sentence = tokenizer.encode_plus(sentence1, sentence_random, max_length=128, pad_to_max_length=True, return_tensors="pt", truncation=True)
encoded_sentence_tuple = encoded_sentence['input_ids'], encoded_sentence['attention_mask'], encoded_sentence['token_type_ids'] 
pickled_bytes = pickle.dumps(encoded_sentence_tuple)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [6]:
%%time
raw_bytes = predictor.predict(pickled_bytes)
print(pickle.loads(raw_bytes))

(tensor([[[ 0.2119, -0.3340,  0.4062,  ...,  0.5469,  0.4531, -0.1196],
         [-0.3457,  0.0942, -0.7969,  ..., -0.1270,  0.9844, -0.4316],
         [-0.0986,  0.3906, -0.5078,  ..., -0.5195,  0.2441, -0.0038],
         ...,
         [-0.0688,  0.1953,  0.4531,  ...,  0.1348, -0.1079, -0.0361],
         [ 0.0271,  0.1387,  0.3887,  ...,  0.0219, -0.1152,  0.0635],
         [-0.5742, -0.2334,  0.1572,  ...,  0.4355,  0.0062,  0.0884]]]), tensor([[ 8.2031e-01,  8.5156e-01,  1.0000e+00, -9.6875e-01, -1.0000e+00,
          7.0703e-01, -9.8047e-01, -6.6406e-01,  1.0000e+00,  1.0000e+00,
          9.9219e-01, -1.0000e+00,  7.8516e-01, -1.0000e+00, -4.3945e-01,
          9.8047e-01,  9.6094e-01,  5.2344e-01, -3.2227e-01,  9.0625e-01,
         -9.1797e-01, -1.0000e+00,  9.9609e-01, -4.8242e-01, -6.9531e-01,
         -1.0000e+00,  9.1406e-01,  3.4961e-01,  3.2422e-01,  9.3262e-02,
          7.6172e-01, -7.1875e-01, -7.0703e-01,  4.3945e-01,  1.0000e+00,
          9.1016e-01, -9.0625e-01,  3.

In [7]:
%%time
total_runs = 5000
endpoint_name = predictor.endpoint
print('Running {} inferences for {}:'.format(total_runs, endpoint_name))

client_times = []
errors_list = []
cw_start = datetime.datetime.utcnow()

errors = 0

for i in range(total_runs):    
    
    
    sentence_random = (random.choice(s_nouns) + ' ' + random.choice(s_verbs) + ' ' + random.choice(s_nouns).lower() or random.choice(p_nouns).lower() + ' ' + random.choice(infinitives))
    encoded_sentence = tokenizer.encode_plus(sentence1, sentence_random, max_length=128, pad_to_max_length=True, return_tensors="pt", truncation=True)
    encoded_sentence_tuple = encoded_sentence['input_ids'], encoded_sentence['attention_mask'], encoded_sentence['token_type_ids'] 
    
    pickled_bytes = pickle.dumps(encoded_sentence_tuple)
    
    client_start = time.time()
    
    try:
        raw_bytes = predictor.predict(pickled_bytes)
        errors_list.append(20)
    except:
        errors += 1
        errors_list.append(30)
        pass
    
    client_end = time.time()
    client_times.append((client_end - client_start)*1000)
    
print('\nErrors - {:.4f} out of {:.4f} total runs | {:.4f}% \n'.format(errors, total_runs, (errors/total_runs)*100))
errors = 0
    
    
cw_end = datetime.datetime.utcnow()    
    
print('Client end-to-end latency percentiles:')
client_avg = np.mean(client_times)
client_p50 = np.percentile(client_times, 50)
client_p90 = np.percentile(client_times, 90)
client_p95 = np.percentile(client_times, 95)
client_p100 = np.percentile(client_times, 100)
print('Avg | P50 | P90 | P95 | P100')
print('{:.4f} | {:.4f} | {:.4f} | {:.4f}\n'.format(client_avg, client_p50, client_p90, client_p95, client_p100))

print('Getting Cloudwatch:')
cloudwatch = boto3.client('cloudwatch')
statistics=['SampleCount', 'Average', 'Minimum', 'Maximum']
extended=['p50', 'p90', 'p95', 'p100']
print('Time elapsed: {} seconds'.format((cw_end - cw_start).total_seconds()))
# Give 5 minute buffer to end
cw_end += datetime.timedelta(minutes=5)

# Period must be 1, 5, 10, 30, or multiple of 60
# Calculate closest multiple of 60 to the total elapsed time
factor = math.ceil((cw_end - cw_start).total_seconds() / 60)
period = factor * 60

print('Using period of {} seconds\n'.format(period))

cloudwatch_ready = False
# Keep polling CloudWatch metrics until datapoints are available
while not cloudwatch_ready:
  time.sleep(30)
  print('Waiting 30 seconds ...')
  # Must use default units of microseconds
  model_latency_metrics = cloudwatch.get_metric_statistics(MetricName='ModelLatency',
                                             Dimensions=[{'Name': 'EndpointName',
                                                          'Value': endpoint_name},
                                                         {'Name': 'VariantName',
                                                          'Value': "AllTraffic"}],
                                             Namespace="AWS/SageMaker",
                                             StartTime=cw_start,
                                             EndTime=cw_end,
                                             Period=period,
                                             Statistics=statistics,
                                             ExtendedStatistics=extended
                                             )
  # Should be 1000
  if len(model_latency_metrics['Datapoints']) > 0:
    print('{} latency datapoints ready'.format(model_latency_metrics['Datapoints'][0]['SampleCount']))
    side_avg = model_latency_metrics['Datapoints'][-1]['Average'] / 1000.0
    side_p50 = model_latency_metrics['Datapoints'][-1]['ExtendedStatistics']['p50'] / 1000.0
    side_p90 = model_latency_metrics['Datapoints'][-1]['ExtendedStatistics']['p90'] / 1000.0
    side_p95 = model_latency_metrics['Datapoints'][-1]['ExtendedStatistics']['p95'] / 1000.0
    side_p100 = model_latency_metrics['Datapoints'][-1]['ExtendedStatistics']['p100'] / 1000.0
    print('Avg | P50 | P90 | P95 | P100')
    print('{:.4f} | {:.4f} | {:.4f} | {:.4f}\n'.format(side_avg, side_p50, side_p90, side_p95, side_p100))

    cloudwatch_ready = True
   


Running 5000 inferences for bert-inf1-serving-2020-12-11-02-17-17-879:

Errors - 0.0000 out of 5000.0000 total runs | 0.0000% 

Client end-to-end latency percentiles:
Avg | P50 | P90 | P95 | P100
29.2387 | 26.9903 | 30.1974 | 31.7383

Getting Cloudwatch:
Time elapsed: 149.294055 seconds
Using period of 480 seconds

Waiting 30 seconds ...
4148.0 latency datapoints ready
Avg | P50 | P90 | P95 | P100
22.1073 | 19.7642 | 22.5034 | 23.9801

CPU times: user 17.5 s, sys: 3.22 s, total: 20.8 s
Wall time: 3min


# Manual Endpoint Load

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role
import boto3 

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [2]:
endpoint_name = 'bert-inf1-serving-2020-12-11-02-17-17-879'
predictor = sagemaker.predictor.RealTimePredictor(endpoint_name)

In [ ]:
predictor.delete_endpoint()